In [ ]:
%%bash
cp ./frozen/ADD.rc .
cp ./frozen/test.config .

# Demo 3: Adding a Customized Instruction to the Ideal ReRAM Recipe Table
In this demonstration, we will show how to add a new recipe to the recipe table. This feature is useful for those who want to perform research evaluating different operations and how they affect the performance of applications. Since we will be modifying the source code, we will need to recompile MASTODON after the modification

### Step 1: Defining the Instruction - Half Adder
In this demo, we will create a new instruction called `HalfADD <A> <B> <C>` that performs the bit-serial half-add operations on input `<A>` and `<B>` and store the sum S back to `<B>` and the carry C to `<C>`
Things to note:
- For half adder, there is no carry-in, carry-out, each bit position simply perform S = A XOR B and C = A AND B
- We need to store S in an intermediate column before writing it back to `<B>` because ReRAM primitives require 3 distinct columns, 2 for inputs, and 1 for output

### Step 2: Modifying ./ReRAM_ideal.h

In [ ]:
# %load ../../src/bottom/ReRAM_ideal.h
#ifndef ReRAM_IDEAL_H
#define ReRAM_IDEAL_H

#include <vector>
#include <deque>
#include <map>
#include <utility>
#include <string>

/*
WARNING: this recipe book represents the \"unrolled\", fully translated, one-shot
recipes that are sent to the back end. In real hardware, the decoder can optionally
go through multiple level of indirection to build each recipe to save memory
space. For example, the MUL8 operation has 3 ADD operations at the end of the
recipe. In real hardware, the decoder will subsitute 3 ADD ops at the end
with the ADD recipes and the appropriate operands to create the unrolled MUL8
recipe. Another example, RDIV8 has a bunch of NOPs in its recipe. In reality, RDIV8
is a super instruction, composed of multiple internal instructions + FLUSH instructions
The FLUSH will replace the chain of NOP in real hardware.

In conclusion, this recipe book represents the final translation right before issuing
to the back end. It does not simulate the intermediate steps to create the recipes.
However, given the cycle-by-cycle translation of the recipe, those intermediate
steps won\'t create extra timing overhead even if we decide to simulate them.

TODO: Maybe in the future we need to simulate these intermediate steps
*/

using namespace std;

class ReRAM_IDEAL
{
public:
  ReRAM_IDEAL(){};
  map <string, deque<string> > get_recipe() {
    map <string, deque<string> > table;

    /* ====================== HalfADD ============================= */
    deque <string> halfadd {{
      "AND A B C",
      "XOR A B i0",
      "COPY i0 B"
    }};
    table.insert({"HalfADD", halfadd});

    /* ====================== LSHIFT ============================= */
    deque <string> lshift {{
      "LOCK",
      "BUF R i0",
      "FILTER 0",
      "COPY c0 i0",
      "UNFILTER",
      "BUF A L",
      "COPY i0 B", // Allow A and B to be the same
      "RELEASE"
    }};
    table.insert({"LSHIFT", lshift});

    /* ====================== MOV ============================= */
    deque <string> mov {{
      "LOCK",
      "NOP",
      "NOP",
      "COPY A B", // Allow A and B to be the same
      "RELEASE"
    }};
    table.insert({"MOV", mov});

    /* ====================== MSB_PRED ============================= */
    deque <string> msb_pred {{
      "LOCK",
      "NOP", // not needed
      "NOP", // not needed
      "BUF A T",
      "RELEASE"
    }};
    table.insert({"MSB_PRED", msb_pred});

    /* ====================== INV_TORUS ============================= */
    deque <string> inv_torus {{
      "FILTER 0",
      "BUF R i0",
      "NOR i0 c0 i1",
      "BUF i1 R",
      "UNFILTER",
      "NOP"
    }};
    table.insert({"INV_TORUS", inv_torus});

    /* ====================== AND ============================= */
    deque <string> binary_and {{
      "LOCK",
      "NOP",
      "AND A B i0",
      "COPY i0 C", // in case C and A are the same
      "RELEASE"
    }};
    table.insert({"AND", binary_and});

    /* ====================== OR ============================= */
    deque <string> binary_or {{
      "LOCK",
      "NOP",
      "OR A B i0",
      "COPY i0 C", // in case C and A are the same
      "RELEASE"
    }};
    table.insert({"OR", binary_or});

    /* ====================== NAND ============================= */
    deque <string> binary_nand {{
      "LOCK",
      "NOP",
      "NAND A B i0",
      "COPY i0 C", // in case C and A are the same
      "RELEASE"
    }};
    table.insert({"NAND", binary_nand});

    /* ====================== NOR ============================= */
    deque <string> binary_nor {{
      "LOCK",
      "NOP",
      "NOR A B i0",
      "COPY i0 C", // in case C and A are the same
      "RELEASE"
    }};
    table.insert({"NOR", binary_nor});

    /* ====================== XOR ============================= */
    deque <string> binary_xor {{
      "LOCK",
      "NOP",
      "XOR A B i0",
      "XOR A B i0",
      "COPY i0 C", // in case C and A are the same
      "RELEASE"
    }};
    table.insert({"XOR", binary_xor});

    /* ====================== XNOR ============================= */
    deque <string> binary_xnor {{
      "LOCK",
      "NOP",
      "XOR A B i0",
      "XOR A B i0",
      "NOR c0 i0 C", // in case C and A are the same
      "RELEASE"
    }};
    table.insert({"XNOR", binary_xnor});

    /* ====================== DINV ============================= */
    deque <string> dinv {{
      "LOCK",
      "NOP",
      "NOR A c0 i0",
      "COPY i0 A",
      "RELEASE"
    }};
    table.insert({"DINV", dinv});

    /* ====================== INV ============================= */
    deque <string> inv {{
      "LOCK",
      "NOP",
      "NOP",
      "NOR A c0 B",
      "RELEASE"
    }};
    table.insert({"INV", inv});

    /* ====================== INIT0 ============================= */
    deque <string> init0 {{
      "LOCK",
      "NOP",
      "NOP",
      "COPY c0 A",
      "RELEASE"
    }};
    table.insert({"INIT0", init0});

    /* ====================== INIT1 ============================= */
    deque <string> init1 {{
      "LOCK",
      "COPY c0 A",
      "FILTER 0",
      "COPY c1 A",
      "UNFILTER",
      "NOP",
      "RELEASE"
    }};
    table.insert({"INIT1", init1});

    /* ====================== INC ============================= */
    // Increment by 1
    deque <string> inc {{
      "LOCK",
      "BUF R i0",
      "FILTER 0",
      "COPY c1 i0",
      "UNFILTER",
      "XOR A i0 i1",
      "XOR A i0 i1",
      "AND A i0 i2",
      "COPY i1 B",
      "BUF i2 L",
      "RELEASE"
    }};
    table.insert({"INC", inc});

    /* ====================== DEC ============================= */
    // Decrement by 1
    deque <string> dec {{
      "LOCK",
      "BUF R i0",
      "FILTER 0",
      "COPY c1 i0",
      "UNFILTER",
      "XOR A i0 i2",
      "XOR A i0 i2",
      "NOR c0 A i4",
      "AND i4 i0 i1",
      "COPY i2 B",
      "BUF i1 L",
      "RELEASE"
    }};
    table.insert({"DEC", dec});

    /* ====================== MUX ============================= */
    deque <string> mux {{
      "LOCK",
      "COPY C i0",
      "NOR c0 i0 i1",
      "AND A i0 i2",
      "AND B i1 i3",
      "OR i2 i3 C",
      "RELEASE"
    }};
    table.insert({"MUX", mux});

    /* ====================== RELUv1 ============================= */
    deque <string> reluv1 {{
      "LOCK",
      "BUF R i0",
      "AND A i0 B",
      "AND A i0 B",
      "BUF i0 L",
      "RELEASE"
    }};
    table.insert({"RELUv1", reluv1});

    /* ====================== ADD ============================= */
    deque <string> add {{
      "LOCK",
      "BUF R i0",
      "XOR A B i1",
      "AND A B i2",
      "XOR i1 i0 C",
      "AND i1 i0 i3",
      "OR i2 i3 i1",
      "BUF i1 L",
      "RELEASE"
    }};
    table.insert({"ADD", add});

    /* ====================== SUB ============================= */
    // Full subtractor: A minus B
    deque <string> sub {{
      "LOCK",
      "BUF R i0",
      "XOR A B i1",
      "XOR A B i1",
      "NOR c0 A i2",
      "AND B i2 i3",
      "XOR i1 i0 C",
      "XOR i1 i0 C",
      "NOR c0 i1 i4",
      "AND i4 i0 i1",
      "OR i3 i1 i4",
      "BUF i4 L",
      "RELEASE",
    }};
    table.insert({"SUB", sub});

    /* ====================== MUL 8 ============================= */
    // TODO c0 is substituting c0 right now, merge request from predicateExecute to use c0
    // i0 - i5 are used to store the shifted version of inputs
    // i7 - i10 are used to store partial products
    // i6, i11 are used to perform AND with NOR
    // At the end, i11 is used to store result of i7 + i8
    // At the end, i6 is used to store result of i9 + i10
    deque <string> mul8 {{
      // Left shift A by 1
      "LOCK",
      "BUF R i0",
      "BUF A L",
      "RELEASE",
      // Left shift A by 2
      "LOCK",
      "BUF R i1",
      "BUF i0 L",
      "RELEASE",
      // Left shift A by 3
      "LOCK",
      "BUF R i2",
      "BUF i1 L",
      "RELEASE",
      // Left shift B by 1
      "LOCK",
      "BUF R i3",
      "BUF B L",
      "RELEASE",
      // Left shift B by 2
      "LOCK",
      "BUF R i4",
      "BUF i3 L",
      "RELEASE",
      // Left shift B by 3
      "LOCK",
      "BUF R i5",
      "BUF i4 L",
      "RELEASE",
      // Generate First Partial Product, store in i7
      "AND A i5 i7",
      "FILTER 0",
      "AND A B i7",
      "UNFILTER",
      "FILTER 1",
      "AND A i3 i7",
      "UNFILTER",
      "FILTER 2",
      "AND A i4 i7",
      "UNFILTER",
      // Generate Second Partial Product, store in i8
      "AND i0 i5 i8",
      "FILTER 1",
      "AND B i0 i8",
      "UNFILTER",
      "FILTER 2",
      "AND i0 i3 i8",
      "UNFILTER",
      "FILTER 3",
      "AND i0 i4 i8",
      "UNFILTER",
      // Generate Third Partial Product, store in i9
      "AND i1 i5 i9",
      "FILTER 2",
      "AND i1 B i9",
      "UNFILTER",
      "FILTER 3",
      "AND i1 i3 i9",
      "UNFILTER",
      "FILTER 4",
      "AND i1 i4 i9",
      "UNFILTER",
      // Generate Fourth Partial Product, store in i10
      "AND i2 i5 i10",
      "FILTER 3",
      "AND B i2 i10",
      "UNFILTER",
      "FILTER 4",
      "AND i2 i3 i10",
      "UNFILTER",
      "FILTER 5",
      "AND i2 i4 i10",
      "UNFILTER",
      // Add i7 to i8 store i11
      "LOCK",
      "BUF R i0",
      "XOR i7 i8 i1",
      "XOR i7 i8 i1", // replicated because of XOR ReRAM behavior require 2 cycl,
      "XOR i1 i0 i11",
      "XOR i1 i0 i11",
      "AND i7 i8 i2",
      "AND i1 i0 i3",
      "OR i2 i3 i1",
      "BUF i1 L",
      "RELEASE",
      // Add i9 to i10 store i6
      "LOCK",
      "BUF R i0",
      "XOR i9 i10 i1",
      "XOR i9 i10 i1", // replicated because of XOR ReRAM behavior require 2 cycl,
      "XOR i1 i0 i6",
      "XOR i1 i0 i6",
      "AND i9 i10 i2",
      "AND i1 i0 i3",
      "OR i2 i3 i1",
      "BUF i1 L",
      "RELEASE",
      // ADD i11 to i6 store in C
      "LOCK",
      "BUF R i0",
      "XOR i11 i6 i1",
      "XOR i11 i6 i1", // replicated because of XOR ReRAM behavior require 2 cycl,
      "XOR i1 i0 C",
      "XOR i1 i0 C",
      "AND i11 i6 i2",
      "AND i1 i0 i3",
      "OR i2 i3 i1",
      "BUF i1 L",
      "RELEASE"
    }};
    table.insert({"MUL8", mul8});

    /* ====================== MAC 8 ============================= */
    // TODO c0 is substituting c0 right now, merge request from predicateExecute to use c0
    // i0 - i5 are used to store the shifted version of inputs
    // i7 - i10 are used to store partial products
    // i6, i11 are used to perform AND with NOR
    // At the end, i11 is used to store result of i7 + i8
    // At the end, i6 is used to store result of i9 + i10
    deque <string> mac8 {{
      // Left shift A by 1
      "LOCK",
      "BUF R i0",
      "BUF A L",
      "RELEASE",
      // Left shift A by 2
      "LOCK",
      "BUF R i1",
      "BUF i0 L",
      "RELEASE",
      // Left shift A by 3
      "LOCK",
      "BUF R i2",
      "BUF i1 L",
      "RELEASE",
      // Left shift B by 1
      "LOCK",
      "BUF R i3",
      "BUF B L",
      "RELEASE",
      // Left shift B by 2
      "LOCK",
      "BUF R i4",
      "BUF i3 L",
      "RELEASE",
      // Left shift B by 3
      "LOCK",
      "BUF R i5",
      "BUF i4 L",
      "RELEASE",
      // Generate First Partial Product, store in i7
      "AND A i5 i7",
      "FILTER 0",
      "AND A B i7",
      "UNFILTER",
      "FILTER 1",
      "AND A i3 i7",
      "UNFILTER",
      "FILTER 2",
      "AND A i4 i7",
      "UNFILTER",
      // Generate Second Partial Product, store in i8
      "AND i0 i5 i8",
      "FILTER 1",
      "AND B i0 i8",
      "UNFILTER",
      "FILTER 2",
      "AND i0 i3 i8",
      "UNFILTER",
      "FILTER 3",
      "AND i0 i4 i8",
      "UNFILTER",
      // Generate Third Partial Product, store in i9
      "AND i1 i5 i9",
      "FILTER 2",
      "AND i1 B i9",
      "UNFILTER",
      "FILTER 3",
      "AND i1 i3 i9",
      "UNFILTER",
      "FILTER 4",
      "AND i1 i4 i9",
      "UNFILTER",
      // Generate Fourth Partial Product, store in i10
      "AND i2 i5 i10",
      "FILTER 3",
      "AND B i2 i10",
      "UNFILTER",
      "FILTER 4",
      "AND i2 i3 i10",
      "UNFILTER",
      "FILTER 5",
      "AND i2 i4 i10",
      "UNFILTER",
      // Add i7 to C store i11
      "LOCK",
      "BUF R i0",
      "XOR i7 C i1",
      "XOR i7 C i1", // replicated because of XOR ReRAM behavior require 2 cycl,
      "XOR i1 i0 i11",
      "XOR i1 i0 i11",
      "AND i7 C i2",
      "AND i1 i0 i3",
      "OR i2 i3 i1",
      "BUF i1 L",
      "RELEASE",
      // Add i11 to i8 store i12
      "LOCK",
      "BUF R i0",
      "XOR i11 i8 i1",
      "XOR i11 i8 i1", // replicated because of XOR ReRAM behavior require 2 cycl,
      "XOR i1 i0 i12",
      "XOR i1 i0 i12",
      "AND i11 i8 i2",
      "AND i1 i0 i3",
      "OR i2 i3 i1",
      "BUF i1 L",
      "RELEASE",
      // Add i9 to i10 store i6
      "LOCK",
      "BUF R i0",
      "XOR i9 i10 i1",
      "XOR i9 i10 i1", // replicated because of XOR ReRAM behavior require 2 cycl,
      "XOR i1 i0 i6",
      "XOR i1 i0 i6",
      "AND i9 i10 i2",
      "AND i1 i0 i3",
      "OR i2 i3 i1",
      "BUF i1 L",
      "RELEASE",
      // ADD i12 to i6 store in C
      "LOCK",
      "BUF R i0",
      "XOR i12 i6 i1",
      "XOR i12 i6 i1", // replicated because of XOR ReRAM behavior require 2 cycl,
      "XOR i1 i0 C",
      "XOR i1 i0 C",
      "AND i12 i6 i2",
      "AND i1 i0 i3",
      "OR i2 i3 i1",
      "BUF i1 L",
      "RELEASE"
    }};
    table.insert({"MAC8", mac8});

     /* ====================== MUL 16 ============================= */
    // TODO c0 is substituting c0 right now, merge request from predicateExecute to use c0
    // i0 - i5 are used to store the shifted version of inputs
    // i7 - i10 are used to store partial products
    // i6, i11 are used to perform AND with NOR
    // At the end, i11 is used to store result of i7 + i8
    // At the end, i6 is used to store result of i9 + i10
    deque <string> mul16 {{
      // Left shift A by 1
      "LOCK",
      "BUF R i0",
      "BUF A L",
      "RELEASE",
      // Left shift A by 2
      "LOCK",
      "BUF R i1",
      "BUF i0 L",
      "RELEASE",
      // Left shift A by 3
      "LOCK",
      "BUF R i2",
      "BUF i1 L",
      "RELEASE",
      // Left shift A by 4
      "LOCK",
      "BUF R i3",
      "BUF i2 L",
      "RELEASE",
      // Left shift A by 5
      "LOCK",
      "BUF R i4",
      "BUF i3 L",
      "RELEASE",
      // Left shift A by 6
      "LOCK",
      "BUF R i5",
      "BUF i4 L",
      "RELEASE",
      // Left shift A by 7
      "LOCK",
      "BUF R i6",
      "BUF i5 L",
      "RELEASE",
      // Left shift B by 1
      "LOCK",
      "BUF R i7",
      "BUF B L",
      "RELEASE",
      // Left shift B by 2
      "LOCK",
      "BUF R i8",
      "BUF i7 L",
      "RELEASE",
      // Left shift B by 3
      "LOCK",
      "BUF R i9",
      "BUF i8 L",
      "RELEASE",
      // Left shift B by 4
      "LOCK",
      "BUF R i10",
      "BUF i9 L",
      "RELEASE",
      // Left shift B by 5
      "LOCK",
      "BUF R i11",
      "BUF i10 L",
      "RELEASE",
      // Left shift B by 6
      "LOCK",
      "BUF R i12",
      "BUF i11 L",
      "RELEASE",
      // Left shift B by 7
      "LOCK",
      "BUF R i13",
      "BUF i12 L",
      "RELEASE",
      // Generate First Partial Product, store in i14
      "AND A i13 i14",
      "FILTER 0",
      "AND A B i14",
      "UNFILTER",
      "FILTER 1",
      "AND A i7 i14",
      "UNFILTER",
      "FILTER 2",
      "AND A i8 i14",
      "UNFILTER",
      "FILTER 3",
      "AND A i9 i14",
      "UNFILTER",
      "FILTER 4",
      "AND A i10 i14",
      "UNFILTER",
      "FILTER 5",
      "AND A i11 i14",
      "UNFILTER",
      "FILTER 6",
      "AND A i12 i14",
      "UNFILTER",
      // Generate Second Partial Product, store in i15
      "AND i0 i13 i15",
      "FILTER 1",
      "AND i0 B i15",
      "UNFILTER",
      "FILTER 2",
      "AND i0 i7 i15",
      "UNFILTER",
      "FILTER 3",
      "AND i0 i8 i15",
      "UNFILTER",
      "FILTER 4",
      "AND i0 i9 i15",
      "UNFILTER",
      "FILTER 5",
      "AND i0 i10 i15",
      "UNFILTER",
      "FILTER 6",
      "AND i0 i11 i15",
      "UNFILTER",
      "FILTER 7",
      "AND i0 i12 i15",
      "UNFILTER",
      // Generate Third Partial Product, store in i16
      "AND i1 i13 i16",
      "FILTER 2",
      "AND i1 B i16",
      "UNFILTER",
      "FILTER 3",
      "AND i1 i7 i16",
      "UNFILTER",
      "FILTER 4",
      "AND i1 i8 i16",
      "UNFILTER",
      "FILTER 5",
      "AND i1 i9 i16",
      "UNFILTER",
      "FILTER 6",
      "AND i1 i10 i16",
      "UNFILTER",
      "FILTER 7",
      "AND i1 i11 i16",
      "UNFILTER",
      "FILTER 8",
      "AND i1 i12 i16",
      "UNFILTER",
      // Generate Fourth Partial Product, store in i17
      "AND i2 i13 i17",
      "FILTER 3",
      "AND i2 B i17",
      "UNFILTER",
      "FILTER 4",
      "AND i2 i7 i17",
      "UNFILTER",
      "FILTER 5",
      "AND i2 i8 i17",
      "UNFILTER",
      "FILTER 6",
      "AND i2 i9 i17",
      "UNFILTER",
      "FILTER 7",
      "AND i2 i10 i17",
      "UNFILTER",
      "FILTER 8",
      "AND i2 i11 i17",
      "UNFILTER",
      "FILTER 9",
      "AND i2 i12 i17",
      "UNFILTER",
      // Generate Fifth Partial Product, store in i18
      "AND i3 i13 i18",
      "FILTER 4",
      "AND i3 B i18",
      "UNFILTER",
      "FILTER 5",
      "AND i3 i7 i18",
      "UNFILTER",
      "FILTER 6",
      "AND i3 i8 i18",
      "UNFILTER",
      "FILTER 7",
      "AND i3 i9 i18",
      "UNFILTER",
      "FILTER 8",
      "AND i3 i10 i18",
      "UNFILTER",
      "FILTER 9",
      "AND i3 i11 i18",
      "UNFILTER",
      "FILTER 10",
      "AND i3 i12 i18",
      "UNFILTER",
      // Generate Sixth Partial Product, store in i19
      "AND i4 i13 i19",
      "FILTER 5",
      "AND i4 B i19",
      "UNFILTER",
      "FILTER 6",
      "AND i4 i7 i19",
      "UNFILTER",
      "FILTER 7",
      "AND i4 i8 i19",
      "UNFILTER",
      "FILTER 8",
      "AND i4 i9 i19",
      "UNFILTER",
      "FILTER 9",
      "AND i4 i10 i19",
      "UNFILTER",
      "FILTER 10",
      "AND i4 i11 i19",
      "UNFILTER",
      "FILTER 11",
      "AND i4 i12 i19",
      "UNFILTER",
      // Generate Seventh Partial Product, store in i20
      "AND i5 i13 i20",
      "FILTER 6",
      "AND i5 B i20",
      "UNFILTER",
      "FILTER 7",
      "AND i5 i7 i20",
      "UNFILTER",
      "FILTER 8",
      "AND i5 i8 i20",
      "UNFILTER",
      "FILTER 9",
      "AND i5 i9 i20",
      "UNFILTER",
      "FILTER 10",
      "AND i5 i10 i20",
      "UNFILTER",
      "FILTER 11",
      "AND i5 i11 i20",
      "UNFILTER",
      "FILTER 12",
      "AND i5 i12 i20",
      "UNFILTER",
      // Generate Eigth Partial Product, store in i21
      "AND i6 i13 i21",
      "FILTER 7",
      "AND i6 B i21",
      "UNFILTER",
      "FILTER 8",
      "AND i6 i7 i21",
      "UNFILTER",
      "FILTER 9",
      "AND i6 i8 i21",
      "UNFILTER",
      "FILTER 10",
      "AND i6 i9 i21",
      "UNFILTER",
      "FILTER 11",
      "AND i6 i10 i21",
      "UNFILTER",
      "FILTER 12",
      "AND i6 i11 i21",
      "UNFILTER",
      "FILTER 13",
      "AND i6 i12 i21",
      "UNFILTER",
      // Add i14 to i15 store i14
      "LOCK",
      "BUF R i0",
      "XOR i14 i15 i1",
      "XOR i14 i15 i1",
      "AND i14 i15 i2",
      "XOR i1 i0 i14",
      "XOR i1 i0 i14",
      "AND i1 i0 i3",
      "OR i2 i3 i1",
      "BUF i1 L",
      "RELEASE",
      // Add i16 to i17 store i16
      "LOCK",
      "BUF R i0",
      "XOR i16 i17 i1",
      "XOR i16 i17 i1",
      "AND i16 i17 i2",
      "XOR i1 i0 i16",
      "XOR i1 i0 i16",
      "AND i1 i0 i3",
      "OR i2 i3 i1",
      "BUF i1 L",
      "RELEASE",
      // Add i18 to i19 store i18
      "LOCK",
      "BUF R i0",
      "XOR i18 i19 i1",
      "XOR i18 i19 i1",
      "AND i18 i19 i2",
      "XOR i1 i0 i18",
      "XOR i1 i0 i18",
      "AND i1 i0 i3",
      "OR i2 i3 i1",
      "BUF i1 L",
      "RELEASE",
      // Add i20 to i21 store i20
      "LOCK",
      "BUF R i0",
      "XOR i20 i21 i1",
      "XOR i20 i21 i1",
      "AND i20 i21 i2",
      "XOR i1 i0 i20",
      "XOR i1 i0 i20",
      "AND i1 i0 i3",
      "OR i2 i3 i1",
      "BUF i1 L",
      "RELEASE",

      // Add i14 to i16 store i14
      "LOCK",
      "BUF R i0",
      "XOR i14 i16 i1",
      "XOR i14 i16 i1",
      "AND i14 i16 i2",
      "XOR i1 i0 i14",
      "XOR i1 i0 i14",
      "AND i1 i0 i3",
      "OR i2 i3 i1",
      "BUF i1 L",
      "RELEASE",
      // Add i18 to i20 store i18
      "LOCK",
      "BUF R i0",
      "XOR i18 i20 i1",
      "XOR i18 i20 i1",
      "AND i18 i20 i2",
      "XOR i1 i0 i18",
      "XOR i1 i0 i18",
      "AND i1 i0 i3",
      "OR i2 i3 i1",
      "BUF i1 L",
      "RELEASE",

      // Final Add
      "LOCK",
      "BUF R i0",
      "XOR i14 i18 i1",
      "XOR i14 i18 i1",
      "AND i14 i18 i2",
      "XOR i1 i0 C",
      "XOR i1 i0 C",
      "AND i1 i0 i3",
      "OR i2 i3 i1",
      "BUF i1 L",
      "RELEASE"
    }};
    table.insert({"MUL16", mul16});

    /* ====================== MUL 32 ============================= */
    // TODO c0 is substituting c0 right now, merge request from predicateExecute to use c0
    // i0 - i5 are used to store the shifted version of inputs
    // i7 - i10 are used to store partial products
    // i6, i11 are used to perform AND with NOR
    // At the end, i11 is used to store result of i7 + i8
    // At the end, i6 is used to store result of i9 + i10
    deque <string> mul32 {{
      // Left shift A by 1
      "LOCK",
      "BUF R i0",
      "BUF A L",
      "RELEASE",
      // Left shift A by 2
      "LOCK",
      "BUF R i1",
      "BUF i0 L",
      "RELEASE",
      // Left shift A by 3
      "LOCK",
      "BUF R i2",
      "BUF i1 L",
      "RELEASE",
      // Left shift A by 4
      "LOCK",
      "BUF R i3",
      "BUF i2 L",
      "RELEASE",
      // Left shift A by 5
      "LOCK",
      "BUF R i4",
      "BUF i3 L",
      "RELEASE",
      // Left shift A by 6
      "LOCK",
      "BUF R i5",
      "BUF i4 L",
      "RELEASE",
      // Left shift A by 7
      "LOCK",
      "BUF R i6",
      "BUF i5 L",
      "RELEASE",
      // Left shift A by 8
      "LOCK",
      "BUF R i7",
      "BUF i6 L",
      "RELEASE",
      // Left shift A by 9
      "LOCK",
      "BUF R i8",
      "BUF i7 L",
      "RELEASE",
      // Left shift A by 10
      "LOCK",
      "BUF R i9",
      "BUF i8 L",
      "RELEASE",
      // Left shift A by 11
      "LOCK",
      "BUF R i10",
      "BUF i9 L",
      "RELEASE",
      // Left shift A by 12
      "LOCK",
      "BUF R i11",
      "BUF i10 L",
      "RELEASE",
      // Left shift A by 13
      "LOCK",
      "BUF R i12",
      "BUF i11 L",
      "RELEASE",
      // Left shift A by 14
      "LOCK",
      "BUF R i13",
      "BUF i12 L",
      "RELEASE",
      // Left shift A by 15
      "LOCK",
      "BUF R i14",
      "BUF i13 L",
      "RELEASE",
      // Left shift B by 1
      "LOCK",
      "BUF R i15",
      "BUF B L",
      "RELEASE",
      // Left shift B by 2
      "LOCK",
      "BUF R i16",
      "BUF i15 L",
      "RELEASE",
      // Left shift B by 3
      "LOCK",
      "BUF R i17",
      "BUF i16 L",
      "RELEASE",
      // Left shift B by 4
      "LOCK",
      "BUF R i18",
      "BUF i17 L",
      "RELEASE",
      // Left shift B by 5
      "LOCK",
      "BUF R i19",
      "BUF i18 L",
      "RELEASE",
      // Left shift B by 6
      "LOCK",
      "BUF R i20",
      "BUF i19 L",
      "RELEASE",
      // Left shift B by 7
      "LOCK",
      "BUF R i21",
      "BUF i20 L",
      "RELEASE",
      // Left shift B by 8
      "LOCK",
      "BUF R i22",
      "BUF i21 L",
      "RELEASE",
      // Left shift B by 9
      "LOCK",
      "BUF R i23",
      "BUF i22 L",
      "RELEASE",
      // Left shift B by 10
      "LOCK",
      "BUF R i24",
      "BUF i23 L",
      "RELEASE",
      // Left shift B by 11
      "LOCK",
      "BUF R i25",
      "BUF i24 L",
      "RELEASE",
      // Left shift B by 12
      "LOCK",
      "BUF R i26",
      "BUF i25 L",
      "RELEASE",
      // Left shift B by 13
      "LOCK",
      "BUF R i27",
      "BUF i26 L",
      "RELEASE",
      // Left shift B by 14
      "LOCK",
      "BUF R i28",
      "BUF i27 L",
      "RELEASE",
      // Left shift B by 15
      "LOCK",
      "BUF R i29",
      "BUF i28 L",
      "RELEASE",
      // Generate 1st Partial Product, store in i30
      "AND i14 B i30",
      "FILTER 0",
      "AND A B i30",
      "UNFILTER",
      "FILTER 1",
      "AND i0 B i30",
      "UNFILTER",
      "FILTER 2",
      "AND i1 B i30",
      "UNFILTER",
      "FILTER 3",
      "AND i2 B i30",
      "UNFILTER",
      "FILTER 4",
      "AND i3 B i30",
      "UNFILTER",
      "FILTER 5",
      "AND i4 B i30",
      "UNFILTER",
      "FILTER 6",
      "AND i5 B i30",
      "UNFILTER",
      "FILTER 7",
      "AND i6 B i30",
      "UNFILTER",
      "FILTER 8",
      "AND i7 B i30",
      "UNFILTER",
      "FILTER 9",
      "AND i8 B i30",
      "UNFILTER",
      "FILTER 10",
      "AND i9 B i30",
      "UNFILTER",
      "FILTER 11",
      "AND i10 B i30",
      "UNFILTER",
      "FILTER 12",
      "AND i11 B i30",
      "UNFILTER",
      "FILTER 13",
      "AND i12 B i30",
      "UNFILTER",
      "FILTER 14",
      "AND i13 B i30",
      "UNFILTER",
      // Generate 2nd Partial Product, store in i31
      "AND i14 i15 i31",
      "FILTER 1",
      "AND A i15 i31",
      "UNFILTER",
      "FILTER 2",
      "AND i0 i15 i31",
      "UNFILTER",
      "FILTER 3",
      "AND i1 i15 i31",
      "UNFILTER",
      "FILTER 4",
      "AND i2 i15 i31",
      "UNFILTER",
      "FILTER 5",
      "AND i3 i15 i31",
      "UNFILTER",
      "FILTER 6",
      "AND i4 i15 i31",
      "UNFILTER",
      "FILTER 7",
      "AND i5 i15 i31",
      "UNFILTER",
      "FILTER 8",
      "AND i6 i15 i31",
      "UNFILTER",
      "FILTER 9",
      "AND i7 i15 i31",
      "UNFILTER",
      "FILTER 10",
      "AND i8 i15 i31",
      "UNFILTER",
      "FILTER 11",
      "AND i9 i15 i31",
      "UNFILTER",
      "FILTER 12",
      "AND i10 i15 i31",
      "UNFILTER",
      "FILTER 13",
      "AND i11 i15 i31",
      "UNFILTER",
      "FILTER 14",
      "AND i12 i15 i31",
      "UNFILTER",
      "FILTER 15",
      "AND i13 i15 i31",
      "UNFILTER",
      // Generate 3(th) Partial Product
      "AND i14 i16 i32",
      "FILTER 2",
      "AND A i16 i32",
      "UNFILTER",
      "FILTER 3",
      "AND i0 i16 i32",
      "UNFILTER",
      "FILTER 4",
      "AND i1 i16 i32",
      "UNFILTER",
      "FILTER 5",
      "AND i2 i16 i32",
      "UNFILTER",
      "FILTER 6",
      "AND i3 i16 i32",
      "UNFILTER",
      "FILTER 7",
      "AND i4 i16 i32",
      "UNFILTER",
      "FILTER 8",
      "AND i5 i16 i32",
      "UNFILTER",
      "FILTER 9",
      "AND i6 i16 i32",
      "UNFILTER",
      "FILTER 10",
      "AND i7 i16 i32",
      "UNFILTER",
      "FILTER 11",
      "AND i8 i16 i32",
      "UNFILTER",
      "FILTER 12",
      "AND i9 i16 i32",
      "UNFILTER",
      "FILTER 13",
      "AND i10 i16 i32",
      "UNFILTER",
      "FILTER 14",
      "AND i11 i16 i32",
      "UNFILTER",
      "FILTER 15",
      "AND i12 i16 i32",
      "UNFILTER",
      "FILTER 16",
      "AND i13 i16 i32",
      "UNFILTER",
      // Generate 4(th) Partial Product
      "AND i14 i17 i33",
      "FILTER 3",
      "AND A i17 i33",
      "UNFILTER",
      "FILTER 4",
      "AND i0 i17 i33",
      "UNFILTER",
      "FILTER 5",
      "AND i1 i17 i33",
      "UNFILTER",
      "FILTER 6",
      "AND i2 i17 i33",
      "UNFILTER",
      "FILTER 7",
      "AND i3 i17 i33",
      "UNFILTER",
      "FILTER 8",
      "AND i4 i17 i33",
      "UNFILTER",
      "FILTER 9",
      "AND i5 i17 i33",
      "UNFILTER",
      "FILTER 10",
      "AND i6 i17 i33",
      "UNFILTER",
      "FILTER 11",
      "AND i7 i17 i33",
      "UNFILTER",
      "FILTER 12",
      "AND i8 i17 i33",
      "UNFILTER",
      "FILTER 13",
      "AND i9 i17 i33",
      "UNFILTER",
      "FILTER 14",
      "AND i10 i17 i33",
      "UNFILTER",
      "FILTER 15",
      "AND i11 i17 i33",
      "UNFILTER",
      "FILTER 16",
      "AND i12 i17 i33",
      "UNFILTER",
      "FILTER 17",
      "AND i13 i17 i33",
      "UNFILTER",
      // Generate 5(th) Partial Product
      "AND i14 i18 i34",
      "FILTER 4",
      "AND A i18 i34",
      "UNFILTER",
      "FILTER 5",
      "AND i0 i18 i34",
      "UNFILTER",
      "FILTER 6",
      "AND i1 i18 i34",
      "UNFILTER",
      "FILTER 7",
      "AND i2 i18 i34",
      "UNFILTER",
      "FILTER 8",
      "AND i3 i18 i34",
      "UNFILTER",
      "FILTER 9",
      "AND i4 i18 i34",
      "UNFILTER",
      "FILTER 10",
      "AND i5 i18 i34",
      "UNFILTER",
      "FILTER 11",
      "AND i6 i18 i34",
      "UNFILTER",
      "FILTER 12",
      "AND i7 i18 i34",
      "UNFILTER",
      "FILTER 13",
      "AND i8 i18 i34",
      "UNFILTER",
      "FILTER 14",
      "AND i9 i18 i34",
      "UNFILTER",
      "FILTER 15",
      "AND i10 i18 i34",
      "UNFILTER",
      "FILTER 16",
      "AND i11 i18 i34",
      "UNFILTER",
      "FILTER 17",
      "AND i12 i18 i34",
      "UNFILTER",
      "FILTER 18",
      "AND i13 i18 i34",
      "UNFILTER",
      // Generate 6(th) Partial Product
      "AND i14 i19 i35",
      "FILTER 5",
      "AND A i19 i35",
      "UNFILTER",
      "FILTER 6",
      "AND i0 i19 i35",
      "UNFILTER",
      "FILTER 7",
      "AND i1 i19 i35",
      "UNFILTER",
      "FILTER 8",
      "AND i2 i19 i35",
      "UNFILTER",
      "FILTER 9",
      "AND i3 i19 i35",
      "UNFILTER",
      "FILTER 10",
      "AND i4 i19 i35",
      "UNFILTER",
      "FILTER 11",
      "AND i5 i19 i35",
      "UNFILTER",
      "FILTER 12",
      "AND i6 i19 i35",
      "UNFILTER",
      "FILTER 13",
      "AND i7 i19 i35",
      "UNFILTER",
      "FILTER 14",
      "AND i8 i19 i35",
      "UNFILTER",
      "FILTER 15",
      "AND i9 i19 i35",
      "UNFILTER",
      "FILTER 16",
      "AND i10 i19 i35",
      "UNFILTER",
      "FILTER 17",
      "AND i11 i19 i35",
      "UNFILTER",
      "FILTER 18",
      "AND i12 i19 i35",
      "UNFILTER",
      "FILTER 19",
      "AND i13 i19 i35",
      "UNFILTER",
      // Generate 7(th) Partial Product
      "AND i14 i20 i36",
      "FILTER 6",
      "AND A i20 i36",
      "UNFILTER",
      "FILTER 7",
      "AND i0 i20 i36",
      "UNFILTER",
      "FILTER 8",
      "AND i1 i20 i36",
      "UNFILTER",
      "FILTER 9",
      "AND i2 i20 i36",
      "UNFILTER",
      "FILTER 10",
      "AND i3 i20 i36",
      "UNFILTER",
      "FILTER 11",
      "AND i4 i20 i36",
      "UNFILTER",
      "FILTER 12",
      "AND i5 i20 i36",
      "UNFILTER",
      "FILTER 13",
      "AND i6 i20 i36",
      "UNFILTER",
      "FILTER 14",
      "AND i7 i20 i36",
      "UNFILTER",
      "FILTER 15",
      "AND i8 i20 i36",
      "UNFILTER",
      "FILTER 16",
      "AND i9 i20 i36",
      "UNFILTER",
      "FILTER 17",
      "AND i10 i20 i36",
      "UNFILTER",
      "FILTER 18",
      "AND i11 i20 i36",
      "UNFILTER",
      "FILTER 19",
      "AND i12 i20 i36",
      "UNFILTER",
      "FILTER 20",
      "AND i13 i20 i36",
      "UNFILTER",
      // Generate 8(th) Partial Product
      "AND i14 i21 i37",
      "FILTER 7",
      "AND A i21 i37",
      "UNFILTER",
      "FILTER 8",
      "AND i0 i21 i37",
      "UNFILTER",
      "FILTER 9",
      "AND i1 i21 i37",
      "UNFILTER",
      "FILTER 10",
      "AND i2 i21 i37",
      "UNFILTER",
      "FILTER 11",
      "AND i3 i21 i37",
      "UNFILTER",
      "FILTER 12",
      "AND i4 i21 i37",
      "UNFILTER",
      "FILTER 13",
      "AND i5 i21 i37",
      "UNFILTER",
      "FILTER 14",
      "AND i6 i21 i37",
      "UNFILTER",
      "FILTER 15",
      "AND i7 i21 i37",
      "UNFILTER",
      "FILTER 16",
      "AND i8 i21 i37",
      "UNFILTER",
      "FILTER 17",
      "AND i9 i21 i37",
      "UNFILTER",
      "FILTER 18",
      "AND i10 i21 i37",
      "UNFILTER",
      "FILTER 19",
      "AND i11 i21 i37",
      "UNFILTER",
      "FILTER 20",
      "AND i12 i21 i37",
      "UNFILTER",
      "FILTER 21",
      "AND i13 i21 i37",
      "UNFILTER",
      // Generate 9(th) Partial Product
      "AND i14 i22 i38",
      "FILTER 8",
      "AND A i22 i38",
      "UNFILTER",
      "FILTER 9",
      "AND i0 i22 i38",
      "UNFILTER",
      "FILTER 10",
      "AND i1 i22 i38",
      "UNFILTER",
      "FILTER 11",
      "AND i2 i22 i38",
      "UNFILTER",
      "FILTER 12",
      "AND i3 i22 i38",
      "UNFILTER",
      "FILTER 13",
      "AND i4 i22 i38",
      "UNFILTER",
      "FILTER 14",
      "AND i5 i22 i38",
      "UNFILTER",
      "FILTER 15",
      "AND i6 i22 i38",
      "UNFILTER",
      "FILTER 16",
      "AND i7 i22 i38",
      "UNFILTER",
      "FILTER 17",
      "AND i8 i22 i38",
      "UNFILTER",
      "FILTER 18",
      "AND i9 i22 i38",
      "UNFILTER",
      "FILTER 19",
      "AND i10 i22 i38",
      "UNFILTER",
      "FILTER 20",
      "AND i11 i22 i38",
      "UNFILTER",
      "FILTER 21",
      "AND i12 i22 i38",
      "UNFILTER",
      "FILTER 22",
      "AND i13 i22 i38",
      "UNFILTER",
      // Generate 10(th) Partial Product
      "AND i14 i23 i39",
      "FILTER 9",
      "AND A i23 i39",
      "UNFILTER",
      "FILTER 10",
      "AND i0 i23 i39",
      "UNFILTER",
      "FILTER 11",
      "AND i1 i23 i39",
      "UNFILTER",
      "FILTER 12",
      "AND i2 i23 i39",
      "UNFILTER",
      "FILTER 13",
      "AND i3 i23 i39",
      "UNFILTER",
      "FILTER 14",
      "AND i4 i23 i39",
      "UNFILTER",
      "FILTER 15",
      "AND i5 i23 i39",
      "UNFILTER",
      "FILTER 16",
      "AND i6 i23 i39",
      "UNFILTER",
      "FILTER 17",
      "AND i7 i23 i39",
      "UNFILTER",
      "FILTER 18",
      "AND i8 i23 i39",
      "UNFILTER",
      "FILTER 19",
      "AND i9 i23 i39",
      "UNFILTER",
      "FILTER 20",
      "AND i10 i23 i39",
      "UNFILTER",
      "FILTER 21",
      "AND i11 i23 i39",
      "UNFILTER",
      "FILTER 22",
      "AND i12 i23 i39",
      "UNFILTER",
      "FILTER 23",
      "AND i13 i23 i39",
      "UNFILTER",
      // Generate 11(th) Partial Product
      "AND i14 i24 i40",
      "FILTER 10",
      "AND A i24 i40",
      "UNFILTER",
      "FILTER 11",
      "AND i0 i24 i40",
      "UNFILTER",
      "FILTER 12",
      "AND i1 i24 i40",
      "UNFILTER",
      "FILTER 13",
      "AND i2 i24 i40",
      "UNFILTER",
      "FILTER 14",
      "AND i3 i24 i40",
      "UNFILTER",
      "FILTER 15",
      "AND i4 i24 i40",
      "UNFILTER",
      "FILTER 16",
      "AND i5 i24 i40",
      "UNFILTER",
      "FILTER 17",
      "AND i6 i24 i40",
      "UNFILTER",
      "FILTER 18",
      "AND i7 i24 i40",
      "UNFILTER",
      "FILTER 19",
      "AND i8 i24 i40",
      "UNFILTER",
      "FILTER 20",
      "AND i9 i24 i40",
      "UNFILTER",
      "FILTER 21",
      "AND i10 i24 i40",
      "UNFILTER",
      "FILTER 22",
      "AND i11 i24 i40",
      "UNFILTER",
      "FILTER 23",
      "AND i12 i24 i40",
      "UNFILTER",
      "FILTER 24",
      "AND i13 i24 i40",
      "UNFILTER",
      // Generate 12(th) Partial Product
      "AND i14 i25 i41",
      "FILTER 11",
      "AND A i25 i41",
      "UNFILTER",
      "FILTER 12",
      "AND i0 i25 i41",
      "UNFILTER",
      "FILTER 13",
      "AND i1 i25 i41",
      "UNFILTER",
      "FILTER 14",
      "AND i2 i25 i41",
      "UNFILTER",
      "FILTER 15",
      "AND i3 i25 i41",
      "UNFILTER",
      "FILTER 16",
      "AND i4 i25 i41",
      "UNFILTER",
      "FILTER 17",
      "AND i5 i25 i41",
      "UNFILTER",
      "FILTER 18",
      "AND i6 i25 i41",
      "UNFILTER",
      "FILTER 19",
      "AND i7 i25 i41",
      "UNFILTER",
      "FILTER 20",
      "AND i8 i25 i41",
      "UNFILTER",
      "FILTER 21",
      "AND i9 i25 i41",
      "UNFILTER",
      "FILTER 22",
      "AND i10 i25 i41",
      "UNFILTER",
      "FILTER 23",
      "AND i11 i25 i41",
      "UNFILTER",
      "FILTER 24",
      "AND i12 i25 i41",
      "UNFILTER",
      "FILTER 25",
      "AND i13 i25 i41",
      "UNFILTER",
      // Generate 13(th) Partial Product
      "AND i14 i26 i42",
      "FILTER 12",
      "AND A i26 i42",
      "UNFILTER",
      "FILTER 13",
      "AND i0 i26 i42",
      "UNFILTER",
      "FILTER 14",
      "AND i1 i26 i42",
      "UNFILTER",
      "FILTER 15",
      "AND i2 i26 i42",
      "UNFILTER",
      "FILTER 16",
      "AND i3 i26 i42",
      "UNFILTER",
      "FILTER 17",
      "AND i4 i26 i42",
      "UNFILTER",
      "FILTER 18",
      "AND i5 i26 i42",
      "UNFILTER",
      "FILTER 19",
      "AND i6 i26 i42",
      "UNFILTER",
      "FILTER 20",
      "AND i7 i26 i42",
      "UNFILTER",
      "FILTER 21",
      "AND i8 i26 i42",
      "UNFILTER",
      "FILTER 22",
      "AND i9 i26 i42",
      "UNFILTER",
      "FILTER 23",
      "AND i10 i26 i42",
      "UNFILTER",
      "FILTER 24",
      "AND i11 i26 i42",
      "UNFILTER",
      "FILTER 25",
      "AND i12 i26 i42",
      "UNFILTER",
      "FILTER 26",
      "AND i13 i26 i42",
      "UNFILTER",
      // Generate 14(th) Partial Product
      "AND i14 i27 i43",
      "FILTER 13",
      "AND A i27 i43",
      "UNFILTER",
      "FILTER 14",
      "AND i0 i27 i43",
      "UNFILTER",
      "FILTER 15",
      "AND i1 i27 i43",
      "UNFILTER",
      "FILTER 16",
      "AND i2 i27 i43",
      "UNFILTER",
      "FILTER 17",
      "AND i3 i27 i43",
      "UNFILTER",
      "FILTER 18",
      "AND i4 i27 i43",
      "UNFILTER",
      "FILTER 19",
      "AND i5 i27 i43",
      "UNFILTER",
      "FILTER 20",
      "AND i6 i27 i43",
      "UNFILTER",
      "FILTER 21",
      "AND i7 i27 i43",
      "UNFILTER",
      "FILTER 22",
      "AND i8 i27 i43",
      "UNFILTER",
      "FILTER 23",
      "AND i9 i27 i43",
      "UNFILTER",
      "FILTER 24",
      "AND i10 i27 i43",
      "UNFILTER",
      "FILTER 25",
      "AND i11 i27 i43",
      "UNFILTER",
      "FILTER 26",
      "AND i12 i27 i43",
      "UNFILTER",
      "FILTER 27",
      "AND i13 i27 i43",
      "UNFILTER",
      // Generate 15(th) Partial Product
      "AND i14 i28 i44",
      "FILTER 14",
      "AND A i28 i44",
      "UNFILTER",
      "FILTER 15",
      "AND i0 i28 i44",
      "UNFILTER",
      "FILTER 16",
      "AND i1 i28 i44",
      "UNFILTER",
      "FILTER 17",
      "AND i2 i28 i44",
      "UNFILTER",
      "FILTER 18",
      "AND i3 i28 i44",
      "UNFILTER",
      "FILTER 19",
      "AND i4 i28 i44",
      "UNFILTER",
      "FILTER 20",
      "AND i5 i28 i44",
      "UNFILTER",
      "FILTER 21",
      "AND i6 i28 i44",
      "UNFILTER",
      "FILTER 22",
      "AND i7 i28 i44",
      "UNFILTER",
      "FILTER 23",
      "AND i8 i28 i44",
      "UNFILTER",
      "FILTER 24",
      "AND i9 i28 i44",
      "UNFILTER",
      "FILTER 25",
      "AND i10 i28 i44",
      "UNFILTER",
      "FILTER 26",
      "AND i11 i28 i44",
      "UNFILTER",
      "FILTER 27",
      "AND i12 i28 i44",
      "UNFILTER",
      "FILTER 28",
      "AND i13 i28 i44",
      "UNFILTER",
      // Generate 16(th) Partial Product
      "AND i14 i29 i45",
      "FILTER 15",
      "AND A i29 i45",
      "UNFILTER",
      "FILTER 16",
      "AND i0 i29 i45",
      "UNFILTER",
      "FILTER 17",
      "AND i1 i29 i45",
      "UNFILTER",
      "FILTER 18",
      "AND i2 i29 i45",
      "UNFILTER",
      "FILTER 19",
      "AND i3 i29 i45",
      "UNFILTER",
      "FILTER 20",
      "AND i4 i29 i45",
      "UNFILTER",
      "FILTER 21",
      "AND i5 i29 i45",
      "UNFILTER",
      "FILTER 22",
      "AND i6 i29 i45",
      "UNFILTER",
      "FILTER 23",
      "AND i7 i29 i45",
      "UNFILTER",
      "FILTER 24",
      "AND i8 i29 i45",
      "UNFILTER",
      "FILTER 25",
      "AND i9 i29 i45",
      "UNFILTER",
      "FILTER 26",
      "AND i10 i29 i45",
      "UNFILTER",
      "FILTER 27",
      "AND i11 i29 i45",
      "UNFILTER",
      "FILTER 28",
      "AND i12 i29 i45",
      "UNFILTER",
      "FILTER 29",
      "AND i13 i29 i45",
      "UNFILTER",
      // Add i30 to i31 store i30
      "LOCK",
      "BUF R i0",
      "XOR i30 i31 i1",
      "XOR i30 i31 i1",
      "AND i30 i31 i2",
      "XOR i1 i0 i30",
      "XOR i1 i0 i30",
      "AND i1 i0 i3",
      "OR i2 i3 i1",
      "BUF i1 L",
      "RELEASE",
      // Add i32 to i33 store i32
      "LOCK",
      "BUF R i0",
      "XOR i32 i33 i1",
      "XOR i32 i33 i1",
      "AND i32 i33 i2",
      "XOR i1 i0 i32",
      "XOR i1 i0 i32",
      "AND i1 i0 i3",
      "OR i2 i3 i1",
      "BUF i1 L",
      "RELEASE",
      // Add i34 to i35 store i34
      "LOCK",
      "BUF R i0",
      "XOR i34 i35 i1",
      "XOR i34 i35 i1",
      "AND i34 i35 i2",
      "XOR i1 i0 i34",
      "XOR i1 i0 i34",
      "AND i1 i0 i3",
      "OR i2 i3 i1",
      "BUF i1 L",
      "RELEASE",
      // Add i36 to i37 store i36
      "LOCK",
      "BUF R i0",
      "XOR i36 i37 i1",
      "XOR i36 i37 i1",
      "AND i36 i37 i2",
      "XOR i1 i0 i36",
      "XOR i1 i0 i36",
      "AND i1 i0 i3",
      "OR i2 i3 i1",
      "BUF i1 L",
      "RELEASE",
      // Add i38 to i39 store i38
      "LOCK",
      "BUF R i0",
      "XOR i38 i39 i1",
      "XOR i38 i39 i1",
      "AND i38 i39 i2",
      "XOR i1 i0 i38",
      "XOR i1 i0 i38",
      "AND i1 i0 i3",
      "OR i2 i3 i1",
      "BUF i1 L",
      "RELEASE",
      // Add i40 to i41 store i40
      "LOCK",
      "BUF R i0",
      "XOR i40 i41 i1",
      "XOR i40 i41 i1",
      "AND i40 i41 i2",
      "XOR i1 i0 i40",
      "XOR i1 i0 i40",
      "AND i1 i0 i3",
      "OR i2 i3 i1",
      "BUF i1 L",
      "RELEASE",
      // Add i42 to i43 store i42
      "LOCK",
      "BUF R i0",
      "XOR i42 i43 i1",
      "XOR i42 i43 i1",
      "AND i42 i43 i2",
      "XOR i1 i0 i42",
      "XOR i1 i0 i42",
      "AND i1 i0 i3",
      "OR i2 i3 i1",
      "BUF i1 L",
      "RELEASE",
      // Add i44 to i45 store i44
      "LOCK",
      "BUF R i0",
      "XOR i44 i45 i1",
      "XOR i44 i45 i1",
      "AND i44 i45 i2",
      "XOR i1 i0 i44",
      "XOR i1 i0 i44",
      "AND i1 i0 i3",
      "OR i2 i3 i1",
      "BUF i1 L",
      "RELEASE",
      // Add i30 to i32 store i30
      "LOCK",
      "BUF R i0",
      "XOR i30 i32 i1",
      "XOR i30 i32 i1",
      "AND i30 i32 i2",
      "XOR i1 i0 i30",
      "XOR i1 i0 i30",
      "AND i1 i0 i3",
      "OR i2 i3 i1",
      "BUF i1 L",
      "RELEASE",
      // Add i34 to i36 store i34
      "LOCK",
      "BUF R i0",
      "XOR i34 i36 i1",
      "XOR i34 i36 i1",
      "AND i34 i36 i2",
      "XOR i1 i0 i34",
      "XOR i1 i0 i34",
      "AND i1 i0 i3",
      "OR i2 i3 i1",
      "BUF i1 L",
      "RELEASE",
      // Add i38 to i40 store i38
      "LOCK",
      "BUF R i0",
      "XOR i38 i40 i1",
      "XOR i38 i40 i1",
      "AND i38 i40 i2",
      "XOR i1 i0 i38",
      "XOR i1 i0 i38",
      "AND i1 i0 i3",
      "OR i2 i3 i1",
      "BUF i1 L",
      "RELEASE",
      // Add i42 to i44 store i42
      "LOCK",
      "BUF R i0",
      "XOR i42 i44 i1",
      "XOR i42 i44 i1",
      "AND i42 i44 i2",
      "XOR i1 i0 i42",
      "XOR i1 i0 i42",
      "AND i1 i0 i3",
      "OR i2 i3 i1",
      "BUF i1 L",
      "RELEASE",
      // Add i30 to i34 store i30
      "LOCK",
      "BUF R i0",
      "XOR i30 i34 i1",
      "XOR i30 i34 i1",
      "AND i30 i34 i2",
      "XOR i1 i0 i30",
      "XOR i1 i0 i30",
      "AND i1 i0 i3",
      "OR i2 i3 i1",
      "BUF i1 L",
      "RELEASE",
      // Add i38 to i42 store i38
      "LOCK",
      "BUF R i0",
      "XOR i38 i42 i1",
      "XOR i38 i42 i1",
      "AND i38 i42 i2",
      "XOR i1 i0 i38",
      "XOR i1 i0 i38",
      "AND i1 i0 i3",
      "OR i2 i3 i1",
      "BUF i1 L",
      "RELEASE",
      // Add i30 to i38 store i30
      "LOCK",
      "BUF R i0",
      "XOR i30 i38 i1",
      "XOR i30 i38 i1",
      "AND i30 i38 i2",
      "XOR i1 i0 C",
      "XOR i1 i0 C",
      "AND i1 i0 i3",
      "OR i2 i3 i1",
      "BUF i1 L",
      "RELEASE",
    }};
    table.insert({"MUL32", mul32});

    /* ====================== CMPLT ============================= */
    deque <string> cmplt {{
      "LOCK",
      "BUF R i0",
      "NOR c0 B i1",
      "AND i1 A i2",
      "XOR A B i3",
      "XOR A B i3",
      "NOR c0 i3 i4",
      "AND i0 i4 i5",
      // "AND R i5 i6",
      "OR i2 i5 i7",
      "BUF i7 T",
      "RELEASE"
    }};
    table.insert({"CMPLT", cmplt});

    /* ====================== CMPST ============================= */
    deque <string> cmpst {{
      "LOCK",
      "BUF R i0",
      "NOR c0 A i1",
      "AND i1 B i2",
      "XOR A B i3",
      "XOR A B i3",
      "NOR c0 i3 i4",
      "AND i0 i4 i5",
      // "AND R i5 i6",
      "OR i2 i5 i7",
      "BUF i7 T",
      "RELEASE"
    }};
    table.insert({"CMPST", cmpst});

    /* ====================== CMPEQ ============================= */
    deque <string> cmpeq {{
      "LOCK",
      "BUF R i0",
      "FILTER 0",
      "COPY c1 i0",
      "UNFILTER",
      "XOR A B i1",
      "XOR A B i1",
      "NOR c0 i1 i2",
      "AND i0 i2 i3",
      "BUF i3 T",
      "RELEASE"
    }};
    table.insert({"CMPEQ", cmpeq});

    /* ====================== FUZZY ============================= */
    deque <string> fuzzy {{
      "LOCK",
      "BUF R i0",
      "FILTER 0",
      "COPY c1 i0",
      "UNFILTER",
      "XOR A B i1",
      "XOR A B i1",
      "NOR c0 i1 i2",
      "AND i0 i2 i3",
      "OR C i3 i1"
      "BUF i1 T",
      "RELEASE"
    }};
    table.insert({"FUZZY", fuzzy});

    /* ====================== BFLIP ============================= */
    // i8 stores the intermediary that can be overwritten
    deque <string> bflip8 {{
      // Left shift A by 1
      "LOCK",
      "BUF R i0",
      "BUF A T",
      "RELEASE",
      // Left shift A by 2
      "LOCK",
      "BUF R i8",
      "BUF i0 T",
      "RELEASE",
      // Left shift A by 3
      "LOCK",
      "BUF R i1",
      "BUF i8 T",
      "RELEASE",
      // Left shift A by 4
      "LOCK",
      "BUF R i8",
      "BUF i1 T",
      "RELEASE",
      // Left shift A by 5
      "LOCK",
      "BUF R i2",
      "BUF i8 T",
      "RELEASE",
      // Left shift A by 6
      "LOCK",
      "BUF R i4",
      "BUF i2 T",
      "RELEASE",
      // Left shift A by 7
      "LOCK",
      "BUF R i3",
      "BUF i4 T",
      "RELEASE",
      // Left shift A by 8 (all 0 in i8)
      "LOCK",
      "BUF R i5",
      "BUF i3 T",
      "RELEASE",
      // Left shift A by 1
      "LOCK",
      "BUF R i8",
      "BUF i5 T",
      "RELEASE",
      // Left shift A by 2
      "LOCK",
      "BUF R i6",
      "BUF i8 T",
      "RELEASE",
      // Left shift A by 3
      "LOCK",
      "BUF R i8",
      "BUF i6 T",
      "RELEASE",
      // Left shift A by 4
      "LOCK",
      "BUF R i7",
      "BUF i8 T",
      "RELEASE",
      // Left shift A by 5
      "LOCK",
      "BUF R i8",
      "BUF i7 T",
      "RELEASE",
      // Latch the correct result
      "FILTER 7",
      "COPY i3 B",
      "UNFILTER",
      "FILTER 6",
      "COPY i2 B",
      "UNFILTER",
      "FILTER 5",
      "COPY i1 B",
      "UNFILTER",
      "FILTER 4",
      "COPY i0 B",
      "UNFILTER",
      "FILTER 3",
      "COPY i7 B",
      "UNFILTER",
      "FILTER 2",
      "COPY i6 B",
      "UNFILTER",
      "FILTER 1",
      "COPY i5 B",
      "UNFILTER",
      "FILTER 0",
      "COPY i4 B",
      "UNFILTER"
    }};
    table.insert({"BFLIP8", bflip8});

    /* ====================== POPC8 ============================= */
    // i0 and i10 stores the intermediary shift value
    deque <string> popc8 {{
      "COPY c0 B", // overwrite B
      // Left shift A by 1
      "LOCK",
      "BUF R i0",
      "BUF A T",
      "RELEASE",
      // Left shift A by 2
      "LOCK",
      "BUF R i10",
      "BUF i0 T",
      "RELEASE",
      // Left shift A by 3
      "LOCK",
      "BUF R i0",
      "BUF i10 T",
      "RELEASE",
      // Left shift A by 4
      "LOCK",
      "BUF R i10",
      "BUF i0 T",
      "RELEASE",
      // Left shift A by 5
      "LOCK",
      "BUF R i0",
      "BUF i10 T",
      "RELEASE",
      // Left shift A by 6, capture the MSB, store in i8
      "LOCK",
      "BUF R i8",
      "BUF i0 T",
      "RELEASE",
      // Capture next bit
      "LOCK",
      "BUF R i7",
      "BUF i8 T",
      "RELEASE",
      // Capture next bit
      "LOCK",
      "BUF R i6",
      "BUF i7 T",
      "RELEASE",
      // Capture next bit
      "LOCK",
      "BUF R i5",
      "BUF i6 T",
      "RELEASE",
      // Capture next bit
      "LOCK",
      "BUF R i4",
      "BUF i5 T",
      "RELEASE",
      // Capture next bit
      "LOCK",
      "BUF R i3",
      "BUF i4 T",
      "RELEASE",
      // Capture next bit
      "LOCK",
      "BUF R i2",
      "BUF i3 T",
      "RELEASE",
      // Capture next bit
      "LOCK",
      "BUF R i1",
      "BUF i2 T",
      "RELEASE",
      "NOP",
      "NOP",
      "NOP",
      "NOP",
      "NOP",
      "NOP",
      "NOP",
      "NOP",
      // Start the shift and add sequence, sum operand store in i9
      "LOCK",
      "BUF R i0",
      "FILTER 0",
      "COPY i8 i0",
      "UNFILTER",
      "XOR B i0 i10",
      "XOR B i0 i10",
      "AND B i0 i11",
      "COPY i10 B",
      "BUF i11 L",
      "RELEASE",
      // Add
      "LOCK",
      "BUF R i0",
      "FILTER 0",
      "COPY i7 i0",
      "UNFILTER",
      "XOR B i0 i10",
      "XOR B i0 i10",
      "AND B i0 i11",
      "COPY i10 B",
      "BUF i11 L",
      "RELEASE",
      // Add
      "LOCK",
      "BUF R i0",
      "FILTER 0",
      "COPY i6 i0",
      "UNFILTER",
      "XOR B i0 i10",
      "XOR B i0 i10",
      "AND B i0 i11",
      "COPY i10 B",
      "BUF i11 L",
      "RELEASE",
      // Add
      "LOCK",
      "BUF R i0",
      "FILTER 0",
      "COPY i5 i0",
      "UNFILTER",
      "XOR B i0 i10",
      "XOR B i0 i10",
      "AND B i0 i11",
      "COPY i10 B",
      "BUF i11 L",
      "RELEASE",
      // Add
      "LOCK",
      "BUF R i0",
      "FILTER 0",
      "COPY i4 i0",
      "UNFILTER",
      "XOR B i0 i10",
      "XOR B i0 i10",
      "AND B i0 i11",
      "COPY i10 B",
      "BUF i11 L",
      "RELEASE",
      // Add
      "LOCK",
      "BUF R i0",
      "FILTER 0",
      "COPY i3 i0",
      "UNFILTER",
      "XOR B i0 i10",
      "XOR B i0 i10",
      "AND B i0 i11",
      "COPY i10 B",
      "BUF i11 L",
      "RELEASE",
      // Add
      "LOCK",
      "BUF R i0",
      "FILTER 0",
      "COPY i2 i0",
      "UNFILTER",
      "XOR B i0 i10",
      "XOR B i0 i10",
      "AND B i0 i11",
      "COPY i10 B",
      "BUF i11 L",
      "RELEASE",
      // Add
      "LOCK",
      "BUF R i0",
      "FILTER 0",
      "COPY i1 i0",
      "UNFILTER",
      "XOR B i0 i10",
      "XOR B i0 i10",
      "AND B i0 i11",
      "COPY i10 B",
      "BUF i11 L",
      "RELEASE"
    }};
    table.insert({"POPC8", popc8});
    /* ====================== RDIV8 ============================= */
    // Remainder division between 8 bit dividend A and 4 bit divisor B
    // First stage dividend store in i11
    // Second stage dvidend input in i3
    // Third stage dividend input in i2
    // Fourth stage dividen input in i1
    deque <string> rdiv8 {{
    "COPY c0 i11",
    // Shift the dividend in the correct place
    // Left shift A by 1
    "LOCK",
    "BUF R i0",
    "BUF A T",
    "RELEASE",
    // Left shift A by 2
    "LOCK",
    "BUF R i10",
    "BUF i0 T",
    "RELEASE",
    // Left shift A by 3
    "LOCK",
    "BUF R i0",
    "BUF i10 T",
    "RELEASE",
    // Left shift A by 4
    "LOCK",
    "BUF R i10",
    "BUF i0 T",
    "RELEASE",
    // Left shift A by 5
    "LOCK",
    "BUF R i0",
    "BUF i10 T",
    "RELEASE",
    // Left shift A by 6, capture the MSB, store in i8
    "LOCK",
    "BUF R i10",
    "BUF i0 T",
    "RELEASE",
    // Capture next bit
    "LOCK",
    "BUF R i0",
    "BUF i10 T",
    "RELEASE",
    // Capture next bit
    "LOCK",
    "BUF R i10",
    "BUF i0 T",
    "RELEASE",
    // Capture next bit
    "LOCK",
    "BUF R i0",
    "BUF i10 T",
    "RELEASE",
    // Capture next bit
    "LOCK",
    "BUF R i4",
    "BUF i0 T",
    "RELEASE",
    // Capture next bit
    "LOCK",
    "BUF R i3",
    "BUF i4 T",
    "RELEASE",
    // Capture next bit
    "LOCK",
    "BUF R i2",
    "BUF i3 T",
    "RELEASE",
    // Capture next bit
    "LOCK",
    "BUF R i1",
    "BUF i2 T",
    "RELEASE",
    // Store the upper half of A in the lower half of i11
    "FILTER 3",
    "COPY i4 i11",
    "UNFILTER",
    "FILTER 2",
    "COPY i4 i11",
    "UNFILTER",
    "FILTER 1",
    "COPY i4 i11",
    "UNFILTER",
    "FILTER 0",
    "COPY i4 i11",
    "UNFILTER",
    // Begin Stage 1 ===============
    // Full subtractor: i11 minus B
    // i7 for borrow in
    // i4 for borrow out
    // i5 stores the potential output pre-MUX
    // i0, i6 as intermediate
    "FILTER 0",
    "COPY c0 R",
    "UNFILTER",
    "LOCK",
    "BUF R i7",
    "XOR i11 B i6",
    "XOR i11 B i6",
    "XOR i7 i6 i5",
    "XOR i7 i6 i5",
    "NOR c0 i6 i4",
    "AND i4 i7 i6",
    "NOR c0 i11 i4",
    "AND i4 B i0",
    "OR i6 i0 i4",
    "BUF i4 T",
    "RELEASE",
    // Create quotient bit
    "FILTER 7",
    "NOR c0 i7 i0",
    "OR A i0 i6",
    "BUF i6 T",
    "UNFILTER",

    "FILTER 0",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "UNFILTER",

    "LOCK",
    "BUF R i6",
    "NOR c0 i6 i7",
    "AND i7 i11 i0",
    "AND i6 i5 i7",
    "OR i7 i0 i10",
    "BUF i6 L",
    "RELEASE",

    "LOCK",
    "BUF R i11",
    "BUF i10 L",
    "RELEASE",

    // Begin stage 2

    "FILTER 0",
    "COPY c0 R",
    "COPY i3 i11",
    "UNFILTER",
    "LOCK",
    "BUF R i7",
    "XOR i11 B i6",
    "XOR i11 B i6",
    "XOR i7 i6 i5",
    "XOR i7 i6 i5",
    "NOR c0 i6 i4",
    "AND i4 i7 i6",
    "NOR c0 i11 i4",
    "AND i4 B i0",
    "OR i6 i0 i4",
    "BUF i4 T",
    "RELEASE",
    // Create quotient bit
    "FILTER 7",
    "NOR c0 i7 i0",
    "OR A i0 i6",
    "BUF i6 T",
    "UNFILTER",

    "FILTER 0",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "UNFILTER",

    "LOCK",
    "BUF R i6",
    "NOR c0 i6 i7",
    "AND i7 i11 i0",
    "AND i6 i5 i7",
    "OR i7 i0 i10",
    "BUF i6 L",
    "RELEASE",

    "LOCK",
    "BUF R i11",
    "BUF i10 L",
    "RELEASE",

    // Begin stage 3

    "FILTER 0",
    "COPY c0 R",
    "COPY i2 i11",
    "UNFILTER",
    "LOCK",
    "BUF R i7",
    "XOR i11 B i6",
    "XOR i11 B i6",
    "XOR i7 i6 i5",
    "XOR i7 i6 i5",
    "NOR c0 i6 i4",
    "AND i4 i7 i6",
    "NOR c0 i11 i4",
    "AND i4 B i0",
    "OR i6 i0 i4",
    "BUF i4 T",
    "RELEASE",
    // Create quotient bit
    "FILTER 7",
    "NOR c0 i7 i0",
    "OR A i0 i6",
    "BUF i6 T",
    "UNFILTER",

    "FILTER 0",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "UNFILTER",

    "LOCK",
    "BUF R i6",
    "NOR c0 i6 i7",
    "AND i7 i11 i0",
    "AND i6 i5 i7",
    "OR i7 i0 i10",
    "BUF i6 L",
    "RELEASE",

    "LOCK",
    "BUF R i11",
    "BUF i10 L",
    "RELEASE",

    // Begin stage 4

    "FILTER 0",
    "COPY c0 R",
    "COPY i1 i11",
    "UNFILTER",
    "LOCK",
    "BUF R i7",
    "XOR i11 B i6",
    "XOR i11 B i6",
    "XOR i7 i6 i5",
    "XOR i7 i6 i5",
    "NOR c0 i6 i4",
    "AND i4 i7 i6",
    "NOR c0 i11 i4",
    "AND i4 B i0",
    "OR i6 i0 i4",
    "BUF i4 T",
    "RELEASE",

    // Create quotient bit
    "FILTER 7",
    "NOR c0 i7 i0",
    "OR A i0 i6",
    "BUF i6 T",
    "UNFILTER",

    "FILTER 0",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "UNFILTER",

    "LOCK",
    "BUF R i6",
    "NOR c0 i6 i7",
    "AND i7 i11 i0",
    "AND i6 i5 i7",
    "OR i7 i0 C",
    "BUF i6 L",
    "RELEASE"
    }};
    table.insert({"RDIV8", rdiv8});

    /* ====================== QRDIV8 ============================= */
    // Remainder division between 8 bit dividend A and 4 bit divisor B
    // First stage dividend store in i11
    // Second stage dvidend input in i3
    // Third stage dividend input in i2
    // Fourth stage dividen input in i1
    deque <string> qrdiv8 {{
    "COPY c0 C",
    "COPY c0 i11",
    // Shift the dividend in the correct place
    // Left shift A by 1
    "LOCK",
    "BUF R i0",
    "BUF A T",
    "RELEASE",
    // Left shift A by 2
    "LOCK",
    "BUF R i10",
    "BUF i0 T",
    "RELEASE",
    // Left shift A by 3
    "LOCK",
    "BUF R i0",
    "BUF i10 T",
    "RELEASE",
    // Left shift A by 4
    "LOCK",
    "BUF R i10",
    "BUF i0 T",
    "RELEASE",
    // Left shift A by 5
    "LOCK",
    "BUF R i0",
    "BUF i10 T",
    "RELEASE",
    // Left shift A by 6, capture the MSB, store in i8
    "LOCK",
    "BUF R i10",
    "BUF i0 T",
    "RELEASE",
    // Capture next bit
    "LOCK",
    "BUF R i0",
    "BUF i10 T",
    "RELEASE",
    // Capture next bit
    "LOCK",
    "BUF R i10",
    "BUF i0 T",
    "RELEASE",
    // Capture next bit
    "LOCK",
    "BUF R i0",
    "BUF i10 T",
    "RELEASE",
    // Capture next bit
    "LOCK",
    "BUF R i4",
    "BUF i0 T",
    "RELEASE",
    // Capture next bit
    "LOCK",
    "BUF R i3",
    "BUF i4 T",
    "RELEASE",
    // Capture next bit
    "LOCK",
    "BUF R i2",
    "BUF i3 T",
    "RELEASE",
    // Capture next bit
    "LOCK",
    "BUF R i1",
    "BUF i2 T",
    "RELEASE",
    // Store the upper half of A in the lower half of i11
    "FILTER 3",
    "COPY i4 i11",
    "UNFILTER",
    "FILTER 2",
    "COPY i4 i11",
    "UNFILTER",
    "FILTER 1",
    "COPY i4 i11",
    "UNFILTER",
    "FILTER 0",
    "COPY i4 i11",
    "UNFILTER",
    // Begin Stage 1 ===============
    // Full subtractor: i11 minus B
    // i7 for borrow in
    // i4 for borrow out
    // i5 stores the potential output pre-MUX
    // i0, i6 as intermediate
    "FILTER 0",
    "COPY c0 R",
    "UNFILTER",
    "LOCK",
    "BUF R i7",
    "XOR i11 B i6",
    "XOR i11 B i6",
    "XOR i7 i6 i5",
    "XOR i7 i6 i5",
    "NOR c0 i6 i4",
    "AND i4 i7 i6",
    "NOR c0 i11 i4",
    "AND i4 B i0",
    "OR i6 i0 i4",
    "BUF i4 T",
    "RELEASE",

    // Create quotient bit
    "FILTER 7",
    "NOR c0 i7 i0",
    "OR A i0 i6",
    "BUF i6 T",
    "UNFILTER",

    "FILTER 0",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "UNFILTER",

    "LOCK",
    "BUF R i6",
    "NOR c0 i6 i7",
    "AND i7 i11 i0",
    "AND i6 i5 i7",
    "OR i7 i0 i10",
    "BUF i6 L",
    "RELEASE",

    "LOCK",
    "BUF R i11",
    "BUF i10 L",
    "RELEASE",

    // latch quotient bit
    "FILTER 3",
    "COPY i6 C",
    "UNFILTER",

    // Begin stage 2

    "FILTER 0",
    "COPY c0 R",
    "COPY i3 i11",
    "UNFILTER",
    "LOCK",
    "BUF R i7",
    "XOR i11 B i6",
    "XOR i11 B i6",
    "XOR i7 i6 i5",
    "XOR i7 i6 i5",
    "NOR c0 i6 i4",
    "AND i4 i7 i6",
    "NOR c0 i11 i4",
    "AND i4 B i0",
    "OR i6 i0 i4",
    "BUF i4 T",
    "RELEASE",
    // Create quotient bit
    "FILTER 7",
    "NOR c0 i7 i0",
    "OR A i0 i6",
    "BUF i6 T",
    "UNFILTER",

    "FILTER 0",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "UNFILTER",

    "LOCK",
    "BUF R i6",
    "NOR c0 i6 i7",
    "AND i7 i11 i0",
    "AND i6 i5 i7",
    "OR i7 i0 i10",
    "BUF i6 L",
    "RELEASE",

    "LOCK",
    "BUF R i11",
    "BUF i10 L",
    "RELEASE",

    // latch quotient bit
    "FILTER 2",
    "COPY i6 C",
    "UNFILTER",

    // Begin stage 3

    "FILTER 0",
    "COPY c0 R",
    "COPY i2 i11",
    "UNFILTER",
    "LOCK",
    "BUF R i7",
    "XOR i11 B i6",
    "XOR i11 B i6",
    "XOR i7 i6 i5",
    "XOR i7 i6 i5",
    "NOR c0 i6 i4",
    "AND i4 i7 i6",
    "NOR c0 i11 i4",
    "AND i4 B i0",
    "OR i6 i0 i4",
    "BUF i4 T",
    "RELEASE",
    // Create quotient bit
    "FILTER 7",
    "NOR c0 i7 i0",
    "OR A i0 i6",
    "BUF i6 T",
    "UNFILTER",

    "FILTER 0",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "UNFILTER",

    "LOCK",
    "BUF R i6",
    "NOR c0 i6 i7",
    "AND i7 i11 i0",
    "AND i6 i5 i7",
    "OR i7 i0 i10",
    "BUF i6 L",
    "RELEASE",

    "LOCK",
    "BUF R i11",
    "BUF i10 L",
    "RELEASE",

    // latch quotient bit
    "FILTER 1",
    "COPY i6 C",
    "UNFILTER",

    // Begin stage 4

    "FILTER 0",
    "COPY c0 R",
    "COPY i1 i11",
    "UNFILTER",
    "LOCK",
    "BUF R i7",
    "XOR i11 B i6",
    "XOR i11 B i6",
    "XOR i7 i6 i5",
    "XOR i7 i6 i5",
    "NOR c0 i6 i4",
    "AND i4 i7 i6",
    "NOR c0 i11 i4",
    "AND i4 B i0",
    "OR i6 i0 i4",
    "BUF i4 T",
    "RELEASE",

    // Create quotient bit
    "FILTER 7",
    "NOR c0 i7 i0",
    "OR A i0 i6",
    "BUF i6 T",
    "UNFILTER",

    "FILTER 0",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "UNFILTER",

    "LOCK",
    "BUF R i6",
    "NOR c0 i6 i7",
    "AND i7 i11 i0",
    "AND i6 i5 i7",
    "OR i7 i0 B",
    "BUF i6 L",
    "RELEASE",

    // latch quotient bit
    "FILTER 0",
    "COPY i6 C",
    "UNFILTER"
    }};
    table.insert({"QRDIV8", qrdiv8});

    /* ====================== QDIV8 ============================= */
    // Remainder division between 8 bit dividend A and 4 bit divisor B
    // First stage dividend store in i11
    // Second stage dvidend input in i3
    // Third stage dividend input in i2
    // Fourth stage dividen input in i1
    deque <string> qdiv8 {{
    "COPY c0 C",
    "COPY c0 i11",
    // Shift the dividend in the correct place
    // Left shift A by 1
    "LOCK",
    "BUF R i0",
    "BUF A T",
    "RELEASE",
    // Left shift A by 2
    "LOCK",
    "BUF R i10",
    "BUF i0 T",
    "RELEASE",
    // Left shift A by 3
    "LOCK",
    "BUF R i0",
    "BUF i10 T",
    "RELEASE",
    // Left shift A by 4
    "LOCK",
    "BUF R i10",
    "BUF i0 T",
    "RELEASE",
    // Left shift A by 5
    "LOCK",
    "BUF R i0",
    "BUF i10 T",
    "RELEASE",
    // Left shift A by 6, capture the MSB, store in i8
    "LOCK",
    "BUF R i10",
    "BUF i0 T",
    "RELEASE",
    // Capture next bit
    "LOCK",
    "BUF R i0",
    "BUF i10 T",
    "RELEASE",
    // Capture next bit
    "LOCK",
    "BUF R i10",
    "BUF i0 T",
    "RELEASE",
    // Capture next bit
    "LOCK",
    "BUF R i0",
    "BUF i10 T",
    "RELEASE",
    // Capture next bit
    "LOCK",
    "BUF R i4",
    "BUF i0 T",
    "RELEASE",
    // Capture next bit
    "LOCK",
    "BUF R i3",
    "BUF i4 T",
    "RELEASE",
    // Capture next bit
    "LOCK",
    "BUF R i2",
    "BUF i3 T",
    "RELEASE",
    // Capture next bit
    "LOCK",
    "BUF R i1",
    "BUF i2 T",
    "RELEASE",
    // Store the upper half of A in the lower half of i11
    "FILTER 3",
    "COPY i4 i11",
    "UNFILTER",
    "FILTER 2",
    "COPY i4 i11",
    "UNFILTER",
    "FILTER 1",
    "COPY i4 i11",
    "UNFILTER",
    "FILTER 0",
    "COPY i4 i11",
    "UNFILTER",
    // Begin Stage 1 ===============
    // Full subtractor: i11 minus B
    // i7 for borrow in
    // i4 for borrow out
    // i5 stores the potential output pre-MUX
    // i0, i6 as intermediate
    "FILTER 0",
    "COPY c0 R",
    "UNFILTER",
    "LOCK",
    "BUF R i7",
    "XOR i11 B i6",
    "XOR i11 B i6",
    "XOR i7 i6 i5",
    "XOR i7 i6 i5",
    "NOR c0 i6 i4",
    "AND i4 i7 i6",
    "NOR c0 i11 i4",
    "AND i4 B i0",
    "OR i6 i0 i4",
    "BUF i4 T",
    "RELEASE",

    // Create quotient bit
    "FILTER 7",
    "NOR c0 i7 i0",
    "OR A i0 i6",
    "BUF i6 T",
    "UNFILTER",

    "FILTER 0",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "UNFILTER",

    "LOCK",
    "BUF R i6",
    "NOR c0 i6 i7",
    "AND i7 i11 i0",
    "AND i6 i5 i7",
    "OR i7 i0 i10",
    "BUF i6 L",
    "RELEASE",

    "LOCK",
    "BUF R i11",
    "BUF i10 L",
    "RELEASE",

    // latch quotient bit
    "FILTER 3",
    "COPY i6 C",
    "UNFILTER",

    // Begin stage 2

    "FILTER 0",
    "COPY c0 R",
    "COPY i3 i11",
    "UNFILTER",
    "LOCK",
    "BUF R i7",
    "XOR i11 B i6",
    "XOR i11 B i6",
    "XOR i7 i6 i5",
    "XOR i7 i6 i5",
    "NOR c0 i6 i4",
    "AND i4 i7 i6",
    "NOR c0 i11 i4",
    "AND i4 B i0",
    "OR i6 i0 i4",
    "BUF i4 T",
    "RELEASE",
    // Create quotient bit
    "FILTER 7",
    "NOR c0 i7 i0",
    "OR A i0 i6",
    "BUF i6 T",
    "UNFILTER",

    "FILTER 0",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "UNFILTER",

    "LOCK",
    "BUF R i6",
    "NOR c0 i6 i7",
    "AND i7 i11 i0",
    "AND i6 i5 i7",
    "OR i7 i0 i10",
    "BUF i6 L",
    "RELEASE",

    "LOCK",
    "BUF R i11",
    "BUF i10 L",
    "RELEASE",

    // latch quotient bit
    "FILTER 2",
    "COPY i6 C",
    "UNFILTER",

    // Begin stage 3

    "FILTER 0",
    "COPY c0 R",
    "COPY i2 i11",
    "UNFILTER",
    "LOCK",
    "BUF R i7",
    "XOR i11 B i6",
    "XOR i11 B i6",
    "XOR i7 i6 i5",
    "XOR i7 i6 i5",
    "NOR c0 i6 i4",
    "AND i4 i7 i6",
    "NOR c0 i11 i4",
    "AND i4 B i0",
    "OR i6 i0 i4",
    "BUF i4 T",
    "RELEASE",
    // Create quotient bit
    "FILTER 7",
    "NOR c0 i7 i0",
    "OR A i0 i6",
    "BUF i6 T",
    "UNFILTER",

    "FILTER 0",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "UNFILTER",

    "LOCK",
    "BUF R i6",
    "NOR c0 i6 i7",
    "AND i7 i11 i0",
    "AND i6 i5 i7",
    "OR i7 i0 i10",
    "BUF i6 L",
    "RELEASE",

    "LOCK",
    "BUF R i11",
    "BUF i10 L",
    "RELEASE",

    // latch quotient bit
    "FILTER 1",
    "COPY i6 C",
    "UNFILTER",

    // Begin stage 4

    "FILTER 0",
    "COPY c0 R",
    "COPY i1 i11",
    "UNFILTER",
    "LOCK",
    "BUF R i7",
    "XOR i11 B i6",
    "XOR i11 B i6",
    "XOR i7 i6 i5",
    "XOR i7 i6 i5",
    "NOR c0 i6 i4",
    "AND i4 i7 i6",
    "NOR c0 i11 i4",
    "AND i4 B i0",
    "OR i6 i0 i4",
    "BUF i4 T",
    "RELEASE",

    // Create quotient bit
    "FILTER 7",
    "NOR c0 i7 i0",
    "OR A i0 i6",
    "BUF i6 T",
    "UNFILTER",

    "FILTER 0",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "NOP",
    "UNFILTER",

    "LOCK",
    "BUF R i6",
    "NOR c0 i6 i7",
    "AND i7 i11 i0",
    "AND i6 i5 i7",
    "OR i7 i0 i10",
    "BUF i6 L",
    "RELEASE",

    // latch quotient bit
    "FILTER 0",
    "COPY i6 C",
    "UNFILTER"
    }};
    table.insert({"QDIV8", qdiv8});

    return table;
  };
};

#endif


### Step 3: Recompiling MASTODON

In [ ]:
%%bash
make

In [ ]:
# %load ADD.rc
COMPUTE 0 0
ADD 1 2 0
COMPUTE_DONE
FLUSH
EOF


In [ ]:
%%bash
./mastodon ./test.config